In [1]:
import pyomo.environ as pyo
import numpy as np
from pyomo.environ import ConcreteModel,Set,RangeSet,Param,Suffix,Reals,NonNegativeReals,NonPositiveReals,Binary,Objective,minimize,maximize,value
from pyomo.core import Constraint,Var,Block,ConstraintList
from pyomo.opt import SolverFactory, SolverStatus, TerminationCondition
import matplotlib.pyplot as plt
from calculations.datahandling import*
from calculations.data_processor import* 



In [2]:
#--Data handling--
#Read of parameters for portfolio
def InputData(data_file):
    inputdata = pd.read_excel(data_file)
    inputdata = inputdata.set_index('Parameter', drop=True)
    inputdata = inputdata.transpose()
    data = {}
    data['hydro'] = inputdata[['Ci', 'yi', 'P_min', 'P_max']].drop('Solar')
    data['solar']=inputdata[['Ci', 'yi', 'P_min', 'P_max']].drop('Hydro1').drop('Hydro2')
    return data

data = InputData('data/Parameters.xlsx')


#Solar production based on forecast 
def read_solar_data(irrad_file, spec_file, start_date, end_date):
    input_data_PV = read_excel_data(spec_file)
    input_data_Irr = read_irr_data(irrad_file)
    PV_power = pv_power_estimated(input_data_PV,input_data_Irr)
    Solar_1=convert_to_dict(PV_power, start_date, end_date, 'H')
    Solar_p=scale_dict(Solar_1, 10)
    return Solar_p

#Start and end dates of the optimization
start_date='2018-05-28 00:00'
end_date='2018-05-28 23:00'

#Original forecast for solar power production
Solar_p=read_solar_data('data/Data_solar_irr_NOR.csv', 'data/PV_spec.xlsx', start_date, end_date)

#Load 
L= {1:30, 2:20, 3:20, 4:30, 5:50, 6:80, 7:50, 8:90, 9:110, 10:150, 11:120, 12:80, 13:70, 14:80, 15:90, 16:160, 17:170, 18:150, 19:120, 20:100, 21:70, 22:60, 23:50, 24:40} 

#--Constants--
Constants= {
    'Load_penalty':100, 
    'Hydro_cap':3000, 
}

# ----------------------Mathematical formulation ------------------------- #
# ----- First stage ------------------ #
def Obj_1st(model):
    return sum(model.yi[i] * model.p[i, j] for i in model.plants for j in model.periods) + sum(model.ki[s] * model.phi[s, j] for s in model.solar for j in model.periods) + sum(model.Li[n] * model.L_p[n,j] for n in model.penalty for j in model.periods) + model.alpha
#def Obj_1st(model):
#    return sum(model.yi[i] * model.p[i, j] for i in model.plants for j in model.periods) + sum(model.ki * model.phi[s, j] for s in model.solar for j in model.periods) + sum(model.Li[n] * model.L_p[n, j] for n in model.penalty for j in model.periods) + model.alpha 
#Production bounds for hydro plants in stage 1
def p_bounds(model,i,j):
    return (model.Pmin[i],model.Pmax[i])   
#Load demand in first stage
def load_rule_FirstStage(model, j):
    return model.p['Hydro1', j] + model.p['Hydro2', j] + model.phi['Solar', j] + model.L_p['Load_penalty',j] == L[j]
#Solar production in stage 1 must be equal to original forecast
def Solar_rule(model,j):
    return  model.phi['Solar',j] == Solar_p[j]
#Sum of hydro production in stage 1 must be lower than available generation
def Hydro_firststage(model,i):
    return sum(model.p['Hydro1',j] + model.p['Hydro2',j] for j in model.periods)<=model.Hmax[i]
def CreateCuts(model,Ci):
    return(model.alpha <= model.Phi[Ci] + sum(model.Lambda[Ci,i]*(model.x[i]-model.x_hat[Ci,i]) for i in model.plants))

#def CreateCuts(m,c):
#    return(m.alpha <= m.Phi[c] + sum(m.Lambda[c,i]*(m.x[i]-m.x_hat[c,i]) for i in m.I)) c = crop cost.... should be similar to how we obtain power produciton cost Ci

# ----- Second-multi stage --------------#
def Obj_2nd(model):
    return sum((model.yi[i])*(model.p_s[i,j] + model.ki[s] * model.phi_s[s, j] + model.Li[n] * model.L_p_s[n,j]) for i in model.plants for s in model.solar for n in model.penalty for j in model.periods)
#Production constraint for hydro1 in stage 2
def hydro1_bounds(model,j):
    return model.p_s['Hydro1',j]<=40    
#Production constraint for hydro2 in stage 2
def hydro2_bounds(model,j):
    return model.p_s['Hydro2',j]<=100
# Constraint to calculate remaining capacity after the first stage
def Remaining_Capacity_rule(model, i):                                                              # LINKING CONSTRAINT!
    return model.Hmax[i] == model.H_max_stage1[i] - sum(model.p[i, j] for j in model.periods)       # LINKING CONSTRAINT!
#Sum of hydro production in stage 2 must be lower than capacity and the already used power in stage 1
def Hydro_secondstage(model,i):
    return sum(model.p_s['Hydro1',j] +model.p_s['Hydro2',j] for j in model.periods)<=model.Hmax[i]-sum(model.p['Hydro1',j] + model.p['Hydro2',j] for j in model.periods)    
#Total power generation in stage 2 must be equal to the hourly set load
def load_rule_TwoStage(model, j):
    return model.p_s['Hydro1',j] +model.p_s['Hydro2',j]+ model.phi_s['Solar', j] + model.L_p_s['Load_penalty',j] == L[j]


# ------------------------- MODEL SETUP -------------------------- #

# Set up model 1st stage
def ModelSetUp_1st(data, Constants,Cuts):
    model = ConcreteModel()

    # Define sets and parameters specific to the first stage
    model.plants = Set(initialize=data['hydro'].index)  # Set of plant types (e.g., 'Hydro1', 'Hydro2')
    model.solar = Set(initialize=data['solar'].index)  # Set of solar types (e.g., 'Solar')
    model.periods = Set(initialize=L.keys())   # Set of time periods (e.g., 1, 2, ..., 24)
    model.penalty= Set(initialize=['Load_penalty']) 
    #Parameters
    model.Li=pyo.Param(model.penalty, initialize=Constants['Load_penalty'])
    #Initial costs for plants
    model.Ci=pyo.Param(model.plants, initialize=data['hydro']['Ci'])
    model.Si=pyo.Param(model.solar, initialize=data['solar']['Ci'])
    #Variable costs for plants 
    model.yi=pyo.Param(model.plants, initialize=data['hydro']['yi'])
    model.ki=pyo.Param(model.solar, initialize=data['solar']['yi'])
    #Production bounds for plants
    model.Pmin=pyo.Param(model.plants, initialize=data['hydro']['P_min'])
    model.Pmax=pyo.Param(model.plants, initialize=data['hydro']['P_max'])
    model.Phi_min=pyo.Param(model.solar, initialize=data['solar']['P_min'])
    model.Phi_max=pyo.Param(model.solar, initialize=data['solar']['P_max'])

    #Maximum hydro production available for the 24 hours
    model.Hmax = pyo.Param(model.plants, initialize=Constants['Hydro_cap'])
    model.H_max_stage1 = pyo.Param(model.plants, initialize=Constants['Hydro_cap'])

    # Load data
    model.L = Param(model.periods, initialize=lambda model, j: L[j])
    # Variables
    model.p = Var(model.plants, model.periods, bounds=p_bounds)  # Production from hydro plants
    model.phi = Var(model.solar, model.periods, within=NonNegativeReals)  # Production from solar installation
    model.L_p = Var(model.penalty, model.periods, within=NonNegativeReals)  # Load penalty

    """Cuts_information"""
    #Set for cuts
    model.Cut = pyo.Set(initialize = Cuts["Set"])
    #Parameter for cuts
    model.Phi = pyo.Param(model.Cut, initialize = Cuts["Phi"])
    model.Lambda = pyo.Param(model.Cut, model.periods, initialize = Cuts["lambda"])
    model.x_hat = pyo.Param(model.Cut, model.periods, initialize = Cuts["x_hat"])
    #Variable for alpha
    model.alpha = pyo.Var(bounds = (-100000000,100000000))
    
    """Constraint cut"""
    model.CreateCuts = pyo.Constraint(model.Cut, rule = CreateCuts)
    model.pbounds = pyo.Constraint(model.plants, model.periods, rule = p_bounds)
    model.load_cons_FirstStage = pyo.Constraint(model.periods, rule=load_rule_FirstStage)
    model.hydro_cons=pyo.Constraint(model.plants, rule=Hydro_firststage)

    """Constraints"""
    # Define objective function
    model.obj = pyo.Objective(rule=Obj_1st, sense=pyo.minimize)
    
    return model

def ModelSetUp_2nd(data, Constants,X_hat, Hmax_stage1):
    # Instance
    model = pyo.ConcreteModel()
    # Define sets
    model.plants = Set(initialize=data['hydro'].index)  # Set of plant types (e.g., 'Hydro1', 'Hydro2')
    model.solar = Set(initialize=data['solar'].index)  # Set of solar types (e.g., 'Solar')
    model.periods = Set(initialize=L.keys()) 
    model.penalty= Set(initialize=['Load_penalty']) 

    # Define parameters
    model.X_hat = pyo.Param(model.periods, initialize = X_hat)
    model.Li=pyo.Param(model.penalty, initialize=Constants['Load_penalty'])
    model.Ci=pyo.Param(model.plants, initialize=data['hydro']['Ci'])
    model.Si=pyo.Param(model.solar, initialize=data['solar']['Ci'])
    model.yi=pyo.Param(model.plants, initialize=data['hydro']['yi'])
    model.ki=pyo.Param(model.solar, initialize=data['solar']['yi'])
    model.Pmin=pyo.Param(model.plants, initialize=data['hydro']['P_min'])
    model.Pmax=pyo.Param(model.plants, initialize=data['hydro']['P_max'])
    model.Phi_min=pyo.Param(model.solar, initialize=data['solar']['P_min'])
    model.Phi_max=pyo.Param(model.solar, initialize=data['solar']['P_max'])
    model.Hmax = pyo.Param(model.plants, initialize=Constants['Hydro_cap'])
    model.H_max_stage1 = pyo.Param(model.plants, initialize=Hmax_stage1)
    
    # Define variables
    model.p_s=pyo.Var(model.plants, model.periods, within=p_bounds)
    model.phi_s=pyo.Var(model.solar, model.periods, within=NonNegativeReals)
    model.L_p_s=pyo.Var(model.penalty, model.periods, within=NonNegativeReals)

    """Constraints"""
    model.load_cons_TwoStage = pyo.Constraint(model.periods, rule=load_rule_TwoStage)
    model.Remaining_Capacity = pyo.Constraint(model.plants, rule=Remaining_Capacity_rule) #To update how much H is left!

    # Define objective function
    model.obj = pyo.Objective(rule=Obj_2nd, sense=pyo.minimize)
    return model


def Solve(model):
    opt = SolverFactory("gurobi")
    model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
    results = opt.solve(model, load_solutions=True)
    return results, model
def DisplayResults(model):
    return print(model.display(), model.dual.display())

# ----------------------- CUTTING ------------------------ #

# Function for creating new linear cuts for optimization problem
def Cut_manage(Cuts,model):
    """Add new cut to existing dictionary of cut information"""
    
    #Find cut iteration by checking number of existing cuts
    cut = len(Cuts["Set"])
    #Add new cut to list, since 0-index is a thing this works well
    Cuts["Set"].append(cut)
    
    #Find 2nd stage cost result
    Cuts["Phi"][cut] = pyo.value(model.obj)
    #Find lambda x_hat for each type of power produced
    for j in model.periods:
        X_hat = {}
        for i in model.plants:
            Cuts["lambda"][cut,i] = model.dual[model.Remaining_Capacity[i]]
            Cuts["x_hat"][cut,i] = model.X_hat[i]
        return(Cuts)

Cuts = {}
Cuts["Set"] = []
Cuts["Phi"] = {}
Cuts["lambda"] = {}
Cuts["x_hat"] = {}

#This is the while-loop in principle, but for this case is only a for-loop
for i in range(10):

    #Solve 1st stage problem
    m_1st = ModelSetUp_1st(data, Constants,Cuts)
    Solve(m_1st)

    # Process 1st stage result
    X_hat = {}
    for j in m_1st.periods:
        X_hat_j = {
            "Hydro1": m_1st.p["Hydro1", j].value,
            "Hydro2": m_1st.p["Hydro2", j].value,
            "Solar": m_1st.phi["Solar", j].value,
            "Load_penalty": m_1st.L_p['Load_penalty', j].value
        }
        X_hat[j] = X_hat_j

    # Pass Hmax_stage1 from the first stage to the second stage
    Hmax_stage1 = {}
    for plant in m_1st.plants:
        Hmax_stage1[plant] = m_1st.Hmax[plant].value

    # Setup and solve 2nd stage problem with updated Hmax_stage1
    m_2nd = ModelSetUp_2nd(data, Constants, X_hat, Hmax_stage1)
    Solve(m_2nd)


    #Create new cuts for 1st stage problem
    Cuts = Cut_manage(Cuts,m_2nd)
    
    #Print results 2nd stage
    print("Objective function:",pyo.value(m_2nd.obj))
    print("Cut information acquired:")
    for component in Cuts:
        if component == "lambda" or component == "x_hat":
            for j in m_2nd.plants:
                print(component,j,Cuts[component][i,j])
        else:
            print(component,Cuts[component])
    input()
    
    #We perform a convergence check
    print("UB:",pyo.value(m_1st.alpha.value),"- LB:",pyo.value(m_2nd.obj))
    input()

def displayresults(model):
    return print(model.display()) #model.dual.display())

model.name="unknown";
    - termination condition: infeasible
    - message from solver: Model was proven to be infeasible.


AttributeError: 'int' object has no attribute 'value'